# Analyzing IMDB Data in Keras

In [1]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

Using TensorFlow backend.


## 1. Loading the data
This dataset comes preloaded with Keras, so one simple command will get us training and testing data. There is a parameter for how many words we want to look at. We've set it at 1000, but feel free to experiment.

In [2]:
# Loading the data (it's preloaded in Keras)
num_words=5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


## 2. Examining the data
Notice that the data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in our dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.

The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [3]:
print(x_train[0])
print(y_train[0])

[1, 11, 4079, 11, 4, 1986, 745, 3304, 299, 1206, 590, 3029, 1042, 37, 47, 27, 1269, 2, 2, 19, 6, 3586, 15, 1367, 3196, 17, 1002, 723, 1768, 2887, 757, 46, 4, 232, 1131, 39, 107, 3589, 11, 4, 4539, 198, 24, 4, 1834, 133, 4, 107, 7, 98, 413, 2, 2, 11, 35, 781, 8, 169, 4, 2179, 5, 259, 334, 3773, 8, 4, 3497, 10, 10, 17, 16, 3381, 46, 34, 101, 612, 7, 84, 18, 49, 282, 167, 2, 2, 122, 24, 1414, 8, 177, 4, 392, 531, 19, 259, 15, 934, 40, 507, 39, 2, 260, 77, 8, 162, 2, 121, 4, 65, 304, 273, 13, 70, 1276, 2, 8, 15, 745, 3304, 5, 27, 322, 2197, 2, 2, 70, 30, 2, 88, 17, 6, 3029, 1042, 29, 100, 30, 4943, 50, 21, 18, 148, 15, 26, 2, 12, 152, 157, 10, 10, 21, 19, 3196, 46, 50, 5, 4, 1636, 112, 828, 6, 1003, 4, 162, 2, 2, 517, 6, 2, 7, 4, 2, 2, 4, 351, 232, 385, 125, 6, 1693, 39, 2383, 5, 29, 69, 2, 2, 6, 162, 2, 1567, 232, 256, 34, 718, 2, 2980, 8, 6, 226, 762, 7, 2, 2, 5, 517, 2, 6, 3242, 7, 4, 351, 232, 37, 9, 1861, 8, 123, 3196, 2, 2, 188, 2, 857, 11, 4, 86, 22, 121, 29, 1990, 1495, 10, 10, 127

## 3. One-hot encoding the output
Here, we'll turn the input vectors into (0,1)-vectors. For example, if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1.

In [4]:
# One-hot encoding the output into vector mode, each of length 50000
tokenizer = Tokenizer(num_words=num_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train[0])

[ 0.  1.  1. ...,  0.  0.  0.]


And we'll also one-hot encode the output.

In [5]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


## 4. Building the  model architecture
Build a model here using sequential. Feel free to experiment with different layers and sizes! Also, experiment adding dropout to reduce overfitting.

In [6]:
# Build the model architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=num_words))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

# Compile the model using a loss function and an optimizer.
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               640128    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 648,514
Trainable params: 648,514
Non-trainable params: 0
_________________________________________________________________


## 5. Training the model
Run the model here. Experiment with different batch_size, and number of epochs!

In [7]:
# Run the model. Feel free to experiment with different batch sizes and number of epochs.
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)

model.fit(x_train, y_train,
          epochs = 12,
          batch_size = 32,
          validation_data = (x_test, y_test),
          verbose = 1,
          callbacks = [checkpointer])

Train on 25000 samples, validate on 25000 samples
Epoch 1/12
25000/25000 [==============================] - 21s 821us/step - loss: 0.3633 - acc: 0.8502 - val_loss: 0.3080 - val_acc: 0.8769
Epoch 2/12
25000/25000 [==============================] - 20s 812us/step - loss: 0.2632 - acc: 0.8997 - val_loss: 0.2981 - val_acc: 0.8793
Epoch 3/12
25000/25000 [==============================] - 20s 809us/step - loss: 0.2378 - acc: 0.9124 - val_loss: 0.2898 - val_acc: 0.8831
Epoch 4/12
25000/25000 [==============================] - 20s 797us/step - loss: 0.2224 - acc: 0.9179 - val_loss: 0.3009 - val_acc: 0.8851
Epoch 5/12
25000/25000 [==============================] - 20s 798us/step - loss: 0.2155 - acc: 0.9212 - val_loss: 0.3182 - val_acc: 0.8775
Epoch 6/12
25000/25000 [==============================] - 19s 778us/step - loss: 0.2128 - acc: 0.9238 - val_loss: 0.3810 - val_acc: 0.8500
Epoch 7/12
25000/25000 [==============================] - 20s 790us/step - loss: 0.2179 - acc: 0.9251 - val_loss: 0.

## 6. Evaluating the model
This will give you the accuracy of the model, as evaluated on the testing set. Can you get something over 85%?

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1]) # 0.86742

Accuracy:  0.8725
